In [1]:
!pip install datasets

   ---------------------------------------- 0.0/480.6 kB ? eta -:--:--
   -- ------------------------------------- 30.7/480.6 kB 1.3 MB/s eta 0:00:01
   --- ----------------------------------- 41.0/480.6 kB 487.6 kB/s eta 0:00:01
   ---------- ----------------------------- 122.9/480.6 kB 1.0 MB/s eta 0:00:01
   ----------------- ---------------------- 204.8/480.6 kB 1.1 MB/s eta 0:00:01
   -------------------------- ------------- 317.4/480.6 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------  471.0/480.6 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 480.6/480.6 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/447.5 kB ? eta -:--:--
   --------------------------------------- 447.5/447.5 kB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/143.5 kB ? eta -:--:


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
      --------------------------------------- 0.3/15.3 MB ? eta -:--:--
     --- ------------------------------------ 1.3/15.3 MB 4.0 MB/s eta 0:00:04
     ------------------- -------------------- 7.6/15.3 MB 14.3 MB/s eta 0:00:01
     ----------------------------------- --- 13.9/15.3 MB 23.0 MB/s eta 0:00:01
     ----------------------------------- --- 13.9/15.3 MB 23.0 MB/s eta 0:00:01
     ----------------------------------- --- 13.9/15.3 MB 23.0 MB/s eta 0:00:01
     ------------------------------------ -- 14.2/15.3 MB 11.1 MB/s eta 0:00:01
     -------------------------------------- - 14.7/15.3 MB 9.6 MB/s eta 0:00:01
     ---------------------------------------  14.9/15.3 MB 8.7 MB/s eta 0:00:01
     ---------------------------------------  15.2/15.3 MB 8.0 MB/s eta 0:00:01
     ---------------------------------------- 15.3/15.3 MB 7.0 MB

In [11]:
!pip install ipywidgets --upgrade




   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -------------------------- ------------- 1.6/2.3 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 9.5 MB/s eta 0:00:00


In [1]:
from typing import List, Tuple

In [31]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import spacy
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from datasets import load_dataset
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [4]:
def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the validation set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    unknown_categories = set(y_test) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the test set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [5]:
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    doc = nlp_pipeline(s)
    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]
    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [6]:
train_data, val_data, test_data, classes_list = load_sib200_ru()

In [7]:
print(f'Categories: {classes_list}')

Categories: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [8]:
print(len(train_data[0]))
print(len(train_data[1]))

701
701


In [9]:
print(len(val_data[0]))
print(len(val_data[1]))

99
99


In [10]:
print(len(test_data[0]))
print(len(test_data[1]))

204
204


In [11]:
nlp = spacy.load('ru_core_news_sm')

In [12]:
print(train_data[0][0])

Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным.


In [13]:
print(normalize_text(train_data[0][0], nlp))

турция с <NUM> сторона окружить морями на запад эгейским на север чёрный и на юг средиземный


In [14]:
print(val_data[0][0])

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


In [15]:
print(normalize_text(val_data[0][0], nlp))

если увеличить расстояние для бег с <NUM> до <NUM> миля скорость становиться не так важный тогда как выносливость превращаться в абсолютный необходимость


In [16]:
print(test_data[0][0])

Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций.


In [17]:
print(normalize_text(test_data[0][0], nlp))

мутация вносить новый генетический вариация в тот время как отбор убирать её из набор проявляться вариация


In [18]:
class_probability = 1.0 / len(classes_list)
max_df = 1.0 - 0.2 * class_probability
print(f'Maximal document frequency of term is {max_df}.')

Maximal document frequency of term is 0.9714285714285714.


In [39]:
classifier = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(
        token_pattern=r'\b\w+\b',  # Токенизация по словам
        max_df=0.9,                # Исключение слишком частых слов
        min_df=2,                  # Исключение слишком редких слов
        sublinear_tf=True,         # Логарифмическое сглаживание
        max_features=10000         # Ограничение числа признаков
    )),
    ('cls', LogisticRegression(
        solver='saga',
        max_iter=500,
        random_state=42,
        class_weight='balanced',
        n_jobs=-1
    ))
])

In [40]:
param_grid = {
    'vectorizer__ngram_range': [(1, 1)],  # Используем только униграммы
    'cls__C': [0.1, 1.0, 10.0],            # Более узкий диапазон C
    'cls__penalty': ['l2'],                 # Используем L2-регуляризацию
    'cls__class_weight': [None, 'balanced']  # Веса классов
}

cv = GridSearchCV(
    estimator=classifier,
    param_grid={
        'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'vectorizer__max_features': [5000, 10000, 20000],  # Различные размеры словаря
        'cls__C': [0.1, 1, 10, 100, 1000],  # Более тонкий диапазон для C
        'cls__penalty': ['l1', 'l2']
    },
    scoring='f1_macro',
    cv=5,
    refit=True,
    n_jobs=-1,
    verbose=2
)

In [41]:
cv.fit([normalize_text(it, nlp) for it in train_data[0]], train_data[1])

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_df=0.9,
                                                        max_features=10000,
                                                        min_df=2,
                                                        sublinear_tf=True,
                                                        token_pattern='\\b\\w+\\b')),
                                       ('cls',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=500,
                                                           n_jobs=-1,
                                                           random_state=42,
                                                           solver='saga'))]),
             n_jobs=-1,
             param_grid={'cls__C': [0.1, 1, 10, 100, 1000],
                         'cls__penalty': ['l1', 'l2'],
                         'vectorizer__max_features': [5000, 10000, 20000],
                         'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             scoring='f1_macro', verbose=2)

In [34]:
print('Best parameters:')
print(cv.best_params_)

Best parameters:
{'cls__C': 100, 'cls__penalty': 'l2', 'vectorizer__ngram_range': (1, 1)}


In [35]:
print('Best F1-macro:')
print(cv.best_score_)

Best F1-macro:
0.6582444570653682


In [36]:
print(f'Vocabulary size is {len(cv.best_estimator_.named_steps["vectorizer"].vocabulary_)}.')

Vocabulary size is 4359.


In [37]:
y_pred = cv.predict([normalize_text(it, nlp) for it in val_data[0]])
print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.80      0.44      0.57         9
         geography       0.67      0.75      0.71         8
            health       1.00      0.55      0.71        11
          politics       0.83      0.71      0.77        14
science/technology       0.61      0.80      0.69        25
            sports       0.75      0.75      0.75        12
            travel       0.59      0.65      0.62        20

          accuracy                           0.69        99
         macro avg       0.75      0.66      0.69        99
      weighted avg       0.72      0.69      0.69        99



In [38]:
y_pred = cv.predict([normalize_text(it, nlp) for it in test_data[0]])
print(classification_report(y_true=test_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.64      0.47      0.55        19
         geography       0.69      0.53      0.60        17
            health       0.50      0.41      0.45        22
          politics       0.77      0.80      0.79        30
science/technology       0.64      0.76      0.70        51
            sports       0.83      0.80      0.82        25
            travel       0.60      0.65      0.63        40

          accuracy                           0.67       204
         macro avg       0.67      0.63      0.65       204
      weighted avg       0.67      0.67      0.66       204

